# 독버섯 vs. 식용버섯 - Multi-layer Neural Network으로 구분하기

### 1.Data Set Description
- http://archive.ics.uci.edu/ml/datasets/Mushroom
- Attribute Information
    + 0 - classes (target attribute): edible=e, poisonous=p
    + 1 - cap-shape: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
    + 2 - cap-surface: fibrous=f, grooves=g, scaly=y, smooth=s
    + 3 - cap-color: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
    + 4 - bruises: bruises=t, no=f
    + 5 - odor: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
    + 6 - gill-attachment: attached=a, descending=d, free=f, notched=n
    + 7 - gill-spacing: close=c,crowded=w,distant=d
    + 8 - gill-size: broad=b, narrow=n
    + 9 - gill-color: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y
    + 10 - stalk-shape: enlarging=e, tapering=t
    + 11 - stalk-root: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=?
    + 12 - stalk-surface-above-ring: fibrous=f, scaly=y, silky=k, smooth=s
    + 13 - stalk-surface-below-ring: fibrous=f, scaly=y, silky=k, smooth=s
    + 14 - stalk-color-above-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
    + 15 - stalk-color-below-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
    + 16 - veil-type: partial=p, universal=u
    + 17 - veil-color: brown=n, orange=o, white=w, yellow=y
    + 18 - ring-number: none=n, one=o, two=t
    + 19 - ring-type: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
    + 20 - spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
    + 21 - population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
    + 22 - habitat: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d

### 2. Pandas DataFame 및 Numpy Array를 사용하여 데이터 준비하기

#### 1) loading Data

In [1]:
import urllib
from scipy import stats
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

path = 'http://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data'
raw_csv = urllib.request.urlopen(path)
col_names = range(23)
df = pd.read_csv(raw_csv, names = col_names)
print(len(df))

8124


In [2]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


#### 2) Categorical Attribute를 Numerical Attribute로 변환

In [3]:
num_columns = df.shape[1]
map_dic = {}
for i in range(num_columns):
    unique_array = df[i].unique()
    N = len(unique_array)
    map_dic[i] = {}
    for j in range(N):
        map_dic[i][unique_array[j]] = j
    df[i] = df[i].map(map_dic[i])

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
2,1,1,0,2,0,2,0,0,1,1,...,0,0,0,0,0,0,0,1,1,2
3,0,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,3,1,3,0,1,1,0,...,0,0,0,0,0,0,1,1,2,1


#### 각 컬럼별 Normalization

In [5]:
for i in range(1, num_columns):
    unique_array = df[i].unique()
    N = len(unique_array)
    map_dic_sub = {}
    for j in range(N):
        if j == 0:
            map_dic_sub[j] = 0
        else:
            map_dic_sub[j] = j / float(N - 1)        
    df[i] = df[i].map(map_dic_sub)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0,0.0,0.000000,0.000000,0.0,0.000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0,0.0,0.0,0.00,0.000,0.0,0.000000
1,1,0.0,0.000000,0.111111,0.0,0.125,0.0,0.0,1.0,0.000000,...,0.0,0.0,0.0,0,0.0,0.0,0.00,0.125,0.2,0.166667
2,1,0.2,0.000000,0.222222,0.0,0.250,0.0,0.0,1.0,0.090909,...,0.0,0.0,0.0,0,0.0,0.0,0.00,0.125,0.2,0.333333
3,0,0.0,0.333333,0.222222,0.0,0.000,0.0,0.0,0.0,0.090909,...,0.0,0.0,0.0,0,0.0,0.0,0.00,0.000,0.0,0.000000
4,1,0.0,0.000000,0.333333,1.0,0.375,0.0,1.0,1.0,0.000000,...,0.0,0.0,0.0,0,0.0,0.0,0.25,0.125,0.4,0.166667


#### 4) 데이터 전처리 작업

In [7]:
data_df = df.iloc[:, 1:]

In [8]:
data_df.head()

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
0,0.0,0.000000,0.000000,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.00,0.000,0.0,0.000000
1,0.0,0.000000,0.111111,0.0,0.125,0.0,0.0,1.0,0.000000,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.00,0.125,0.2,0.166667
2,0.2,0.000000,0.222222,0.0,0.250,0.0,0.0,1.0,0.090909,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.00,0.125,0.2,0.333333
3,0.0,0.333333,0.222222,0.0,0.000,0.0,0.0,0.0,0.090909,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.00,0.000,0.0,0.000000
4,0.0,0.000000,0.333333,1.0,0.375,0.0,1.0,1.0,0.000000,1.0,...,0.0,0.0,0.0,0,0.0,0.0,0.25,0.125,0.4,0.166667


In [9]:
data_array = data_df.as_matrix()

In [10]:
print(type(data_array), data_array.shape)

<class 'numpy.ndarray'> (8124, 22)


In [11]:
label_df = df.iloc[:, 0:1]

In [12]:
label_df.head()

,0
0,0
1,1
2,1
3,0
4,1


In [13]:
label_array = label_df.as_matrix()

In [14]:
print(type(label_array), label_array.shape)

<class 'numpy.ndarray'> (8124, 1)


In [15]:
label_array_one_hot = []
for i in label_array:
    if i == 0:
        one_hot = [1, 0]
    else:
        one_hot = [0, 1]
    label_array_one_hot.append(one_hot)   

In [16]:
label_array_one_hot =  np.array(label_array_one_hot)

In [17]:
print(type(label_array_one_hot), label_array_one_hot.shape)

<class 'numpy.ndarray'> (8124, 2)


In [18]:
train = {'data': None, 'labels': None}
validation = {'data': None, 'labels': None}
test = {'data': None, 'labels': None}

In [19]:
import random
pick = random.random()
train_data = []
train_labels = []
validation_data = []
validation_labels = []
test_data = []
test_labels = []
for i in range(len(data_array)):
    pick = random.random()
    if pick < 0.80:
        train_data.append(data_array[i])
        train_labels.append(label_array_one_hot[i])
    elif pick >= 0.80 and pick < 0.85:
        validation_data.append(data_array[i])
        validation_labels.append(label_array_one_hot[i])
    elif pick >= 0.85:
        test_data.append(data_array[i])
        test_labels.append(label_array_one_hot[i])
        
train['data'] = np.array(train_data)
train['labels'] = np.array(train_labels)
validation['data'] = np.array(validation_data)
validation['labels'] = np.array(validation_labels)
test['data'] = np.array(test_data)
test['labels'] = np.array(test_labels)

In [20]:
print(train['data'].shape, train['labels'].shape)

(6494, 22) (6494, 2)


In [21]:
print(train['data'])
print(train['labels'])

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.11111111 ...,  0.125       0.2         0.16666667]
 [ 0.2         0.          0.22222222 ...,  0.125       0.2         0.33333333]
 ..., 
 [ 0.          0.          0.         ...,  1.          0.6         1.        ]
 [ 0.8         0.33333333  0.         ...,  0.5         0.6         1.        ]
 [ 0.          0.          0.         ...,  0.75        1.          1.        ]]
[[1 0]
 [0 1]
 [0 1]
 ..., 
 [0 1]
 [1 0]
 [0 1]]


In [22]:
print(validation['data'].shape, validation['labels'].shape)

(395, 22) (395, 2)


In [23]:
print(validation['data'])
print(validation['labels'])

[[ 0.2         0.          0.11111111 ...,  0.          0.          0.33333333]
 [ 0.          0.33333333  0.11111111 ...,  0.125       0.2         0.33333333]
 [ 0.4         0.66666667  0.33333333 ...,  0.125       0.6         0.        ]
 ..., 
 [ 0.8         0.33333333  0.44444444 ...,  0.5         0.6         1.        ]
 [ 0.8         0.33333333  0.         ...,  0.5         0.6         1.        ]
 [ 0.          0.          0.         ...,  0.75        0.6         1.        ]]
[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 

In [24]:
print(test['data'].shape, test['labels'].shape)

(1235, 22) (1235, 2)


In [25]:
print(test['data'])
print(test['labels'])

[[ 0.          0.33333333  0.11111111 ...,  0.125       0.2         0.16666667]
 [ 0.4         0.66666667  0.33333333 ...,  0.125       0.8         0.        ]
 [ 0.2         0.          0.11111111 ...,  0.125       0.2         0.33333333]
 ..., 
 [ 0.2         0.          0.22222222 ...,  0.5         0.2         0.16666667]
 [ 0.8         0.33333333  0.         ...,  0.5         0.6         1.        ]
 [ 0.6         0.          0.         ...,  1.          1.          1.        ]]
[[0 1]
 [0 1]
 [0 1]
 ..., 
 [0 1]
 [1 0]
 [0 1]]


### 3. Multi-layer Neural Network 모델 생성 및 테스트 하기
- homework

In [26]:
import tensorflow as tf

##### 3 Layer Neural Network
- Hyper 파라미터 정의
- input shape : 22
- output shape : 2
- hidden layer size : 128

- learning rate = 0.05

In [27]:
# define hyperparameter
INPUT_SIZE = 22
HIDDEN_SIZE = 128
CLASSES = 2

Learning_Rate = 0.05

- 입력 데이터를 받기 위한 placeholder 정의

In [28]:
# batch size, data size
x = tf.placeholder( tf.float32, shape=[None, INPUT_SIZE] )
y_ = tf.placeholder( tf.float32, shape=[None, CLASSES] )

- mapping 정보에 대한 변수

In [29]:
tensor_map = {x: train['data'], y_: train['labels']}

- Network 구성

In [30]:
W_h1 = tf.Variable(tf.truncated_normal(shape=[INPUT_SIZE, HIDDEN_SIZE]), dtype=tf.float32);
b_h1 = tf.Variable(tf.zeros(shape=[HIDDEN_SIZE]), dtype=tf.float32)

#W_h2 = tf.Variable(tf.truncated_normal(shape=[HIDDEN1_SIZE, HIDDEN2_SIZE]), dtype=tf.float32);
#b_h2 = tf.Variable(tf.zeros(shape=[HIDDEN2_SIZE]), dtype=tf.float32)

W_out = tf.Variable(tf.truncated_normal(shape=[HIDDEN_SIZE, CLASSES]), dtype=tf.float32);
b_out = tf.Variable(tf.zeros(shape=[CLASSES]), dtype=tf.float32)

In [31]:
hidden = tf.sigmoid(tf.matmul(x, W_h1) + b_h1)

#hidden2 = tf.sigmoid(tf.matmul(hidden1, W_h2) + b_h2)

#y = tf.nn.softmax(tf.matmul(hidden, W_out) + b_out)
y = tf.matmul(hidden, W_out) + b_out

- cost function 정의
- 학습 방법(optimizer) 선택

In [32]:
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train = tf.train.GradientDescentOptimizer(Learning_Rate).minimize(cross_entropy)

- training 시작

In [33]:
sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

for i in range(15000):
    _, train_loss = sess.run([train, cross_entropy], feed_dict=tensor_map)
    validation_loss = sess.run(cross_entropy, feed_dict={x: validation['data'], y_: validation['labels']})
    if i % 1000 == 0:
        print ("Step: ", i)
        print("train_loss: ", train_loss)
        print("validation_loss: ", validation_loss)

Step:  0
train_loss:  2.57142
validation_loss:  1.58073
Step:  1000
train_loss:  0.0933776
validation_loss:  0.0947215
Step:  2000
train_loss:  0.0616554
validation_loss:  0.0678787
Step:  3000
train_loss:  0.0464394
validation_loss:  0.0543011
Step:  4000
train_loss:  0.0376256
validation_loss:  0.0456805
Step:  5000
train_loss:  0.031635
validation_loss:  0.0393669
Step:  6000
train_loss:  0.027168
validation_loss:  0.0343644
Step:  7000
train_loss:  0.0236624
validation_loss:  0.0302418
Step:  8000
train_loss:  0.0208352
validation_loss:  0.0267926
Step:  9000
train_loss:  0.0185218
validation_loss:  0.023902
Step:  10000
train_loss:  0.0166115
validation_loss:  0.0214868
Step:  11000
train_loss:  0.0150206
validation_loss:  0.0194712
Step:  12000
train_loss:  0.0136838
validation_loss:  0.0177827
Step:  13000
train_loss:  0.0125494
validation_loss:  0.0163579
Step:  14000
train_loss:  0.0115773
validation_loss:  0.0151435


In [34]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [35]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

- 결과

In [36]:
print(sess.run(accuracy, feed_dict={x: test['data'], y_: test['labels']}))

0.99919


In [37]:
sess.close()

##### 3 Layer Neural Network
- Hyper 파라미터 정의
- input shape : 22
- output shape : 2
- hidden layer size : 128

- learning rate = 0.05

In [38]:
import tensorflow as tf

# define hyperparameter
INPUT_SIZE = 22
HIDDEN1_SIZE = 128
HIDDEN2_SIZE = 128
CLASSES = 2

Learning_Rate = 0.05

In [39]:
# batch size, data size
x = tf.placeholder( tf.float32, shape=[None, INPUT_SIZE] )
y_ = tf.placeholder( tf.float32, shape=[None, CLASSES] )

In [40]:
W_h1 = tf.Variable(tf.truncated_normal(shape=[INPUT_SIZE, HIDDEN_SIZE]), dtype=tf.float32);
b_h1 = tf.Variable(tf.zeros(shape=[HIDDEN_SIZE]), dtype=tf.float32)

W_h2 = tf.Variable(tf.truncated_normal(shape=[HIDDEN1_SIZE, HIDDEN2_SIZE]), dtype=tf.float32);
b_h2 = tf.Variable(tf.zeros(shape=[HIDDEN2_SIZE]), dtype=tf.float32)

W_out = tf.Variable(tf.truncated_normal(shape=[HIDDEN2_SIZE, CLASSES]), dtype=tf.float32);
b_out = tf.Variable(tf.zeros(shape=[CLASSES]), dtype=tf.float32)

In [41]:
hidden1 = tf.sigmoid(tf.matmul(x, W_h1) + b_h1)

hidden2 = tf.sigmoid(tf.matmul(hidden1, W_h2) + b_h2)

#y = tf.nn.softmax(tf.matmul(hidden, W_out) + b_out)
y = tf.matmul(hidden2, W_out) + b_out

In [42]:
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train = tf.train.GradientDescentOptimizer(Learning_Rate).minimize(cross_entropy)

In [43]:
sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

for i in range(15000):
    _, train_loss = sess.run([train, cross_entropy], feed_dict=tensor_map)
    validation_loss = sess.run(cross_entropy, feed_dict={x: validation['data'], y_: validation['labels']})
    if i % 1000 == 0:
        print ("Step: ", i)
        print("train_loss: ", train_loss)
        print("validation_loss: ", validation_loss)

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]
	 [[Node: Mean_2/_11 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_114_Mean_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_2', defined at:
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Program Files\Anaconda3\lib\site-packages\traitlets\config\application.py", line 653, in launch_inst

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: test['data'], y_: test['labels']}))

In [ ]:
sess.close();